In [8]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-14'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']
    return date




In [9]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])

In [10]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [11]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [12]:
drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,23,0,39091000.0,39281000.0,39015000.0,107.932931
1,22,30,39265000.0,39265000.0,38671000.0,576.524574
2,22,0,38747000.0,38820000.0,38245000.0,458.301380
3,21,30,38556000.0,39070000.0,38426000.0,426.664918
4,21,0,39063000.0,39070000.0,38500000.0,510.425343


In [13]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
ups[ups<0] =0
downs[downs>0] = 0
period = 14 
au = ups.abs().ewm(com = period-1, min_periods = period).mean() 
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)
df['RSI'] = RSI

0      45.256034
1      46.977929
2      40.745658
3      38.260991
4      42.671355
         ...    
378          NaN
379          NaN
380          NaN
381          NaN
382          NaN
Name: tradePrice, Length: 383, dtype: float64


In [15]:
# CCI
sm = 0
N = 20
D = 0
for k in range(10):
    for j in range(N):
        for i in range(N):
            M = ( df['tradePrice'][k+i] + df['lowPrice'][k+i] + df['highPrice'][k+i] ) / 3
            sm += M
        sm = sm/N
        M = ( df['tradePrice'][k+j] + df['lowPrice'][k+j] + df['highPrice'][k+j] ) / 3
        D += ( M - sm )
    D = D/N
    for i in range(N):
        M = ( df['tradePrice'][i] + df['lowPrice'][i] + df['highPrice'][i] ) / 3
        sm += M
    sm = sm/N
    M = ( df['tradePrice'][k] + df['lowPrice'][k] + df['highPrice'][k] ) / 3
    CCI = (M - sm) / (0.015 * D)
    print(CCI)



69.91632503628325
65.16629581708762
79.0563109666165
76.55958629442529
70.6290932094041
84.82262413632785
92.20519245273823
78.41851343252011
67.7321428519809
61.69913319765782
